# Análise de Indicadores de Marketing - BAP 2025

Este notebook contém análises dos dados de marketing extraídos da aba Marketing do arquivo KPI - 2025 BAP.xlsx

## Estrutura dos Dados

O arquivo `KPI_Marketing_Preparado.xlsx` contém:
- **6 tabelas principais** (uma por aba)
- **Resumo_Analitico**: Estatísticas descritivas de todas as tabelas
- **Dados_Consolidados_Long**: Todos os dados em formato long/tidy para análises

### Tabelas disponíveis:
1. **Marketing_Geral**: Métricas gerais (Instagram, visualizações, custos de Ads, alcance)
2. **Leads_Condominios**: Origens de propostas enviadas e leads convertidos
3. **Indices_Condominios**: KPIs financeiros (CAC, MRR, ROI, custo por proposta)
4. **Campanha_Imoveis**: Performance da campanha de imóveis (investimento, leads, conversões)
5. **Campanha_Boleto_Digital**: Adoção do boleto digital (unidades, economia)
6. **Campanha_Multiseguros**: Performance da campanha de seguros

## 1. Configuração e Importação de Dados

In [ ]:
# Importar bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Configurações de visualização
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

# Configurar pandas para exibir mais colunas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

print("[OK] Bibliotecas importadas com sucesso!")

In [ ]:
# Carregar dados preparados
file_path = Path('KPI_Marketing_Preparado.xlsx')

if not file_path.exists():
    print("ERRO: Execute primeiro o script 'preparar_dados_marketing.py'")
else:
    # Carregar cada tabela
    df_marketing_geral = pd.read_excel(file_path, sheet_name='Marketing_Geral')
    df_leads = pd.read_excel(file_path, sheet_name='Leads_Condominios')
    df_indices = pd.read_excel(file_path, sheet_name='Indices_Condominios')
    df_imoveis = pd.read_excel(file_path, sheet_name='Campanha_Imoveis')
    df_boleto = pd.read_excel(file_path, sheet_name='Campanha_Boleto_Digital')
    df_seguros = pd.read_excel(file_path, sheet_name='Campanha_Multiseguros')
    df_resumo = pd.read_excel(file_path, sheet_name='Resumo_Analitico')
    df_consolidado = pd.read_excel(file_path, sheet_name='Dados_Consolidados_Long')
    
    print("[OK] Dados carregados com sucesso!")
    print(f"\nTotal de registros no formato consolidado: {len(df_consolidado)}")

## 2. Visão Geral dos Dados

In [ ]:
# Resumo de preenchimento das tabelas
print("=" * 70)
print("RESUMO DE PREENCHIMENTO DAS TABELAS")
print("=" * 70)
df_resumo

In [ ]:
# Visualizar preenchimento
fig, ax = plt.subplots(figsize=(12, 6))
ax.barh(df_resumo['Tabela'], df_resumo['Pct_Preenchimento'], color='steelblue')
ax.set_xlabel('Percentual de Preenchimento (%)', fontsize=12)
ax.set_ylabel('Tabela', fontsize=12)
ax.set_title('Percentual de Preenchimento por Tabela', fontsize=14, fontweight='bold')
ax.set_xlim(0, 100)

# Adicionar valores nas barras
for i, v in enumerate(df_resumo['Pct_Preenchimento']):
    ax.text(v + 1, i, f'{v:.1f}%', va='center')

plt.tight_layout()
plt.show()

## 3. Análise de Marketing Geral

In [ ]:
# Visualizar dados de marketing geral
print("=" * 70)
print("MARKETING GERAL - MÉTRICAS")
print("=" * 70)
df_marketing_geral

In [ ]:
# Preparar dados para visualização
meses_cols = ['Janeiro', 'Fevereiro', 'março', 'abril', 'maio', 'junho', 
              'julho', 'agosto', 'setembro', 'outubro', 'novembro', 'dezembro']

# Encontrar colunas de meses que existem (case insensitive)
meses_disponiveis = [col for col in df_marketing_geral.columns if any(mes.lower() in col.lower() for mes in ['janeiro', 'fevereiro', 'março', 'marco', 'abril', 'maio', 'junho', 'julho', 'agosto', 'setembro', 'outubro', 'novembro', 'dezembro'])]

print(f"Meses encontrados: {meses_disponiveis}")

In [ ]:
# Gráfico de evolução - Novos Seguidores Instagram
metrica_col = df_marketing_geral.columns[0]  # Coluna de métrica
seguidores_idx = df_marketing_geral[df_marketing_geral[metrica_col].str.contains('Seguidores', case=False, na=False)].index[0]
dados_seguidores = df_marketing_geral.loc[seguidores_idx, meses_disponiveis]

fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(range(len(dados_seguidores)), dados_seguidores.values, marker='o', linewidth=2, markersize=8, color='#e74c3c')
ax.fill_between(range(len(dados_seguidores)), dados_seguidores.values, alpha=0.3, color='#e74c3c')
ax.set_xticks(range(len(meses_disponiveis)))
ax.set_xticklabels(meses_disponiveis, rotation=45, ha='right')
ax.set_ylabel('Novos Seguidores', fontsize=12)
ax.set_title('Evolução de Novos Seguidores - Instagram (2025)', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Estatísticas
print(f"\nMédia mensal: {dados_seguidores.mean():.0f} novos seguidores")
print(f"Máximo: {dados_seguidores.max():.0f} ({meses_disponiveis[dados_seguidores.argmax()]})")
print(f"Mínimo: {dados_seguidores.min():.0f} ({meses_disponiveis[dados_seguidores.argmin()]})")

In [ ]:
# Análise de custo de Ads e alcance
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Gráfico 1: Custo de Ads
custo_ads_idx = df_marketing_geral[df_marketing_geral[metrica_col].str.contains('Custo geral de Ads', case=False, na=False)].index[0]
dados_custo = df_marketing_geral.loc[custo_ads_idx, meses_disponiveis]

axes[0].bar(range(len(dados_custo)), dados_custo.values, color='#3498db', alpha=0.8)
axes[0].set_xticks(range(len(meses_disponiveis)))
axes[0].set_xticklabels(meses_disponiveis, rotation=45, ha='right')
axes[0].set_ylabel('Custo (R$)', fontsize=12)
axes[0].set_title('Investimento em Ads por Mês', fontsize=13, fontweight='bold')
axes[0].grid(True, alpha=0.3, axis='y')

# Adicionar valores nas barras
for i, v in enumerate(dados_custo.values):
    if not np.isnan(v):
        axes[0].text(i, v, f'R${v:,.0f}', ha='center', va='bottom', fontsize=9)

# Gráfico 2: Alcance Orgânico vs Pago
alcance_org_idx = df_marketing_geral[df_marketing_geral[metrica_col].str.contains('Alcance Orgânico', case=False, na=False)].index
alcance_pago_idx = df_marketing_geral[df_marketing_geral[metrica_col].str.contains('Alcance Pago', case=False, na=False)].index

if len(alcance_org_idx) > 0 and len(alcance_pago_idx) > 0:
    dados_org = df_marketing_geral.loc[alcance_org_idx[0], meses_disponiveis]
    dados_pago = df_marketing_geral.loc[alcance_pago_idx[0], meses_disponiveis]
    
    x = np.arange(len(meses_disponiveis))
    width = 0.35
    
    axes[1].bar(x - width/2, dados_org.values, width, label='Orgânico', color='#2ecc71', alpha=0.8)
    axes[1].bar(x + width/2, dados_pago.values, width, label='Pago', color='#f39c12', alpha=0.8)
    axes[1].set_xticks(x)
    axes[1].set_xticklabels(meses_disponiveis, rotation=45, ha='right')
    axes[1].set_ylabel('Alcance', fontsize=12)
    axes[1].set_title('Alcance Orgânico vs Pago', fontsize=13, fontweight='bold')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print(f"\nInvestimento total em Ads: R$ {dados_custo.sum():,.2f}")
print(f"Investimento médio mensal: R$ {dados_custo.mean():,.2f}")

## 4. Análise de Leads - Condominios

In [ ]:
# Visualizar estrutura dos dados de leads
print("=" * 70)
print("LEADS - CONDOMINIOS")
print("=" * 70)
print(f"Total de métricas: {len(df_leads)}\n")
df_leads.head(10)

In [ ]:
# Análise de origens de propostas enviadas
metrica_col_leads = df_leads.columns[0]
propostas_mask = df_leads[metrica_col_leads].str.contains('proposta enviada', case=False, na=False)
df_propostas = df_leads[propostas_mask].copy()

print("Origens de Propostas Enviadas:")
print(df_propostas[[metrica_col_leads]].to_string(index=False))

# Calcular totais por origem
meses_leads = [col for col in df_propostas.columns if col != metrica_col_leads]
df_propostas['Total'] = df_propostas[meses_leads].sum(axis=1)

# Gráfico de pizza - Distribuição de origens
fig, ax = plt.subplots(figsize=(10, 8))

# Extrair nomes das origens (remover prefixo)
origens = df_propostas[metrica_col_leads].str.replace('Origem da proposta enviada', '').str.strip(' -').values
totais = df_propostas['Total'].values

# Filtrar apenas valores não-zero
mask_nonzero = totais > 0
origens_filtradas = origens[mask_nonzero]
totais_filtrados = totais[mask_nonzero]

colors = sns.color_palette('husl', len(origens_filtradas))
wedges, texts, autotexts = ax.pie(totais_filtrados, labels=origens_filtradas, autopct='%1.1f%%',
                                    startangle=90, colors=colors, textprops={'fontsize': 10})
ax.set_title('Distribuição de Propostas por Origem (Total 2025)', fontsize=14, fontweight='bold')

# Melhorar legibilidade
for autotext in autotexts:
    autotext.set_color('white')
    autotext.set_fontweight('bold')

plt.tight_layout()
plt.show()

print(f"\nTotal de propostas enviadas: {totais_filtrados.sum():.0f}")

In [ ]:
# Taxa de conversão por origem
convertidos_mask = df_leads[metrica_col_leads].str.contains('Lead Convertido', case=False, na=False)
df_convertidos = df_leads[convertidos_mask].copy()

# Calcular totais
df_convertidos['Total'] = df_convertidos[meses_leads].sum(axis=1)

# Comparar propostas enviadas vs convertidas
print("\n=" * 70)
print("ANÁLISE DE CONVERSÃO")
print("=" * 70)

# Mapear origens comuns
origens_map = {
    'Indicação': 'Indica',
    'Capt. Ativa': 'Capt. Ativa',
    'Capt. Receptiva': 'Contato Receptivo',
    'Construtora': 'Construtora',
    'Reativação': 'Reativa',
    'Ads': 'Ads',
    'Mala Direta': 'Mala Direta'
}

conversao_data = []
for origem_convertido, origem_proposta in origens_map.items():
    # Buscar dados de propostas
    prop_row = df_propostas[df_propostas[metrica_col_leads].str.contains(origem_proposta, case=False, na=False)]
    # Buscar dados de convertidos
    conv_row = df_convertidos[df_convertidos[metrica_col_leads].str.contains(origem_convertido, case=False, na=False)]
    
    if len(prop_row) > 0 and len(conv_row) > 0:
        total_prop = prop_row['Total'].values[0]
        total_conv = conv_row['Total'].values[0]
        taxa = (total_conv / total_prop * 100) if total_prop > 0 else 0
        conversao_data.append({
            'Origem': origem_convertido,
            'Propostas': total_prop,
            'Convertidos': total_conv,
            'Taxa_Conversao_%': taxa
        })

df_conversao = pd.DataFrame(conversao_data)
df_conversao = df_conversao.sort_values('Taxa_Conversao_%', ascending=False)
print(df_conversao.to_string(index=False))

In [ ]:
# Gráfico de taxa de conversão por origem
fig, ax = plt.subplots(figsize=(12, 6))

bars = ax.barh(df_conversao['Origem'], df_conversao['Taxa_Conversao_%'], color='#9b59b6', alpha=0.8)
ax.set_xlabel('Taxa de Conversão (%)', fontsize=12)
ax.set_ylabel('Origem do Lead', fontsize=12)
ax.set_title('Taxa de Conversão por Origem de Lead', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3, axis='x')

# Adicionar valores
for i, (bar, row) in enumerate(zip(bars, df_conversao.itertuples())):
    width = bar.get_width()
    ax.text(width + 0.5, bar.get_y() + bar.get_height()/2, 
            f'{row.Taxa_Conversao__:.1f}% ({int(row.Convertidos)}/{int(row.Propostas)})',
            va='center', fontsize=9)

plt.tight_layout()
plt.show()

## 5. Análise de Índices (KPIs) - Condominios

In [ ]:
# Visualizar índices
print("=" * 70)
print("ÍNDICES - CONDOMINIOS (KPIs)")
print("=" * 70)
df_indices

In [ ]:
# Evolução do CAC (Custo de Aquisição de Cliente)
metrica_col_indices = df_indices.columns[0]
meses_indices = [col for col in df_indices.columns if col != metrica_col_indices]

cac_idx = df_indices[df_indices[metrica_col_indices].str.contains('CAC', case=False, na=False)].index[0]
dados_cac = df_indices.loc[cac_idx, meses_indices]

fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(range(len(dados_cac)), dados_cac.values, marker='o', linewidth=2.5, markersize=10, color='#e67e22')
ax.fill_between(range(len(dados_cac)), dados_cac.values, alpha=0.2, color='#e67e22')
ax.set_xticks(range(len(meses_indices)))
ax.set_xticklabels(meses_indices, rotation=45, ha='right')
ax.set_ylabel('CAC (R$)', fontsize=12)
ax.set_title('Evolução do Custo de Aquisição de Cliente (CAC)', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)

# Linha de média
media_cac = dados_cac.mean()
ax.axhline(y=media_cac, color='red', linestyle='--', linewidth=2, label=f'Média: R$ {media_cac:,.2f}')
ax.legend()

plt.tight_layout()
plt.show()

print(f"\nCAC Médio: R$ {dados_cac.mean():,.2f}")
print(f"CAC Mínimo: R$ {dados_cac.min():,.2f} ({meses_indices[dados_cac.argmin()]})")
print(f"CAC Máximo: R$ {dados_cac.max():,.2f} ({meses_indices[dados_cac.argmax()]})")

In [ ]:
# MRR (Monthly Recurring Revenue) e ROI
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Gráfico 1: MRR
mrr_idx = df_indices[df_indices[metrica_col_indices].str.contains('MRR', case=False, na=False)].index
if len(mrr_idx) > 0:
    dados_mrr = df_indices.loc[mrr_idx[0], meses_indices]
    axes[0].bar(range(len(dados_mrr)), dados_mrr.values, color='#27ae60', alpha=0.8)
    axes[0].set_xticks(range(len(meses_indices)))
    axes[0].set_xticklabels(meses_indices, rotation=45, ha='right')
    axes[0].set_ylabel('MRR (R$)', fontsize=12)
    axes[0].set_title('Monthly Recurring Revenue (MRR)', fontsize=13, fontweight='bold')
    axes[0].grid(True, alpha=0.3, axis='y')
    
    # Adicionar valores
    for i, v in enumerate(dados_mrr.values):
        if not np.isnan(v):
            axes[0].text(i, v, f'R${v/1000:.1f}k', ha='center', va='bottom', fontsize=9)

# Gráfico 2: ROI (Recorrente mensal / Custo)
roi_idx = df_indices[df_indices[metrica_col_indices].str.contains('Recorrente mensal / Custo', case=False, na=False)].index
if len(roi_idx) > 0:
    dados_roi = df_indices.loc[roi_idx[0], meses_indices]
    
    colors_roi = ['#27ae60' if v > 1 else '#e74c3c' for v in dados_roi.values]
    axes[1].bar(range(len(dados_roi)), dados_roi.values, color=colors_roi, alpha=0.8)
    axes[1].axhline(y=1, color='black', linestyle='--', linewidth=2, label='Break-even (ROI=1)')
    axes[1].set_xticks(range(len(meses_indices)))
    axes[1].set_xticklabels(meses_indices, rotation=45, ha='right')
    axes[1].set_ylabel('ROI (Razão)', fontsize=12)
    axes[1].set_title('ROI: Receita Recorrente / Custo de Aquisição', fontsize=13, fontweight='bold')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3, axis='y')
    
    # Adicionar valores
    for i, v in enumerate(dados_roi.values):
        if not np.isnan(v):
            axes[1].text(i, v, f'{v:.2f}x', ha='center', va='bottom' if v > 0 else 'top', fontsize=9)

plt.tight_layout()
plt.show()

if len(mrr_idx) > 0:
    print(f"\nMRR Médio: R$ {dados_mrr.mean():,.2f}")
    print(f"MRR Total (ano): R$ {dados_mrr.sum():,.2f}")
if len(roi_idx) > 0:
    print(f"\nROI Médio: {dados_roi.mean():.2f}x")
    print(f"Meses com ROI positivo (>1): {(dados_roi > 1).sum()}/{len(dados_roi.dropna())}")

## 6. Comparação entre Campanhas

In [ ]:
# Comparar investimento das 3 campanhas
print("=" * 70)
print("COMPARAÇÃO DE CAMPANHAS")
print("=" * 70)

# Extrair dados de investimento
meses_camp = [col for col in df_imoveis.columns if col not in ['Métrica', 'M�trica']]

inv_imoveis = df_imoveis[df_imoveis.iloc[:, 0].str.contains('Investimento', case=False, na=False)].iloc[0, 1:].values
inv_seguros = df_seguros[df_seguros.iloc[:, 0].str.contains('Investimento', case=False, na=False)].iloc[0, 1:].values

# Nota: Boleto Digital não tem investimento direto, mas tem economia

fig, ax = plt.subplots(figsize=(14, 6))

x = np.arange(len(meses_camp))
width = 0.35

ax.bar(x - width/2, inv_imoveis, width, label='Campanha Imóveis', color='#3498db', alpha=0.8)
ax.bar(x + width/2, inv_seguros, width, label='Campanha Multiseguros', color='#e74c3c', alpha=0.8)

ax.set_xlabel('Mês', fontsize=12)
ax.set_ylabel('Investimento (R$)', fontsize=12)
ax.set_title('Comparação de Investimento entre Campanhas', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(meses_camp, rotation=45, ha='right')
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print(f"\nInvestimento total Imóveis: R$ {np.nansum(inv_imoveis):,.2f}")
print(f"Investimento total Seguros: R$ {np.nansum(inv_seguros):,.2f}")

In [ ]:
# Comparar ROI das campanhas
roi_imoveis = df_imoveis[df_imoveis.iloc[:, 0].str.contains('ROI', case=False, na=False)].iloc[0, 1:].values
roi_seguros = df_seguros[df_seguros.iloc[:, 0].str.contains('ROI', case=False, na=False)].iloc[0, 1:].values

fig, ax = plt.subplots(figsize=(14, 6))

ax.plot(range(len(roi_imoveis)), roi_imoveis, marker='o', linewidth=2, markersize=8, label='Imóveis', color='#3498db')
ax.plot(range(len(roi_seguros)), roi_seguros, marker='s', linewidth=2, markersize=8, label='Seguros', color='#e74c3c')

ax.set_xticks(range(len(meses_camp)))
ax.set_xticklabels(meses_camp, rotation=45, ha='right')
ax.set_ylabel('ROI (%)', fontsize=12)
ax.set_title('Comparação de ROI entre Campanhas', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nROI médio Imóveis: {np.nanmean(roi_imoveis):.2f}%")
print(f"ROI médio Seguros: {np.nanmean(roi_seguros):.2f}%")

## 7. Análise de Boleto Digital

In [ ]:
# Evolução da adoção do boleto digital
print("=" * 70)
print("CAMPANHA BOLETO DIGITAL")
print("=" * 70)
df_boleto

In [ ]:
# Gráfico de adoção e economia
meses_boleto = [col for col in df_boleto.columns if col not in ['Métrica', 'M�trica']]

unidades = df_boleto[df_boleto.iloc[:, 0].str.contains('N° de Unidades', case=False, na=False)].iloc[0, 1:].values
economia = df_boleto[df_boleto.iloc[:, 0].str.contains('Economia', case=False, na=False)].iloc[0, 1:].values
pct_base = df_boleto[df_boleto.iloc[:, 0].str.contains('% da base', case=False, na=False)].iloc[0, 1:].values * 100

fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Gráfico 1: Unidades aderidas
axes[0].bar(range(len(unidades)), unidades, color='#16a085', alpha=0.8)
axes[0].set_xticks(range(len(meses_boleto)))
axes[0].set_xticklabels(meses_boleto, rotation=45, ha='right')
axes[0].set_ylabel('Número de Unidades', fontsize=12)
axes[0].set_title('Unidades com Boleto Digital', fontsize=13, fontweight='bold')
axes[0].grid(True, alpha=0.3, axis='y')

# Gráfico 2: Economia gerada
axes[1].bar(range(len(economia)), economia, color='#27ae60', alpha=0.8)
axes[1].set_xticks(range(len(meses_boleto)))
axes[1].set_xticklabels(meses_boleto, rotation=45, ha='right')
axes[1].set_ylabel('Economia (R$)', fontsize=12)
axes[1].set_title('Economia com Impressão (R$ 0,32 por unidade)', fontsize=13, fontweight='bold')
axes[1].grid(True, alpha=0.3, axis='y')

# Adicionar valores
for i, v in enumerate(economia):
    if not np.isnan(v):
        axes[1].text(i, v, f'R${v:.0f}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

print(f"\nTotal de unidades aderidas: {np.nansum(unidades):.0f}")
print(f"Economia total gerada: R$ {np.nansum(economia):,.2f}")
print(f"Percentual da base atual: {np.nanmax(pct_base):.2f}%")

## 8. Análise usando Dados Consolidados (Formato Long)

In [ ]:
# Explorar dados consolidados
print("=" * 70)
print("DADOS CONSOLIDADOS (FORMATO LONG)")
print("=" * 70)
print(f"Total de registros: {len(df_consolidado)}\n")
print("Tabelas disponíveis:")
print(df_consolidado['Tabela'].value_counts())
print("\nPreview:")
df_consolidado.head(10)

In [ ]:
# Análise temporal agregada - Média de valores por mês (todas as métricas)
df_consolidado['Valor_Abs'] = df_consolidado['Valor'].abs()
valores_por_mes = df_consolidado.groupby('Mês_Num')['Valor'].agg(['count', 'mean', 'sum']).reset_index()

print("Estatísticas por mês:")
print(valores_por_mes)

## 9. Insights e Recomendações

### Principais Insights:

1. **Marketing Geral**:
   - Acompanhar evolução de seguidores e correlacionar com investimento em Ads
   - Analisar eficiência de alcance pago vs orgânico

2. **Leads - Condominios**:
   - Identificar canais com melhor taxa de conversão
   - Otimizar investimento priorizando origens mais eficientes

3. **Índices (KPIs)**:
   - Monitorar CAC para mantê-lo abaixo da média histórica
   - Garantir que MRR/Custo (ROI) permaneça > 1 para sustentabilidade

4. **Campanhas**:
   - Comparar ROI entre campanhas para priorizar investimentos
   - Campanha Boleto Digital: baixo custo e economia recorrente

### Próximos Passos:
- Criar dashboard interativo (Power BI, Tableau, Streamlit)
- Implementar alertas para KPIs fora da meta
- Análise preditiva para projeção de MRR e CAC
- Segmentação avançada de leads por características

## 10. Exportar Dados para Análises Externas

In [ ]:
# Exportar dados consolidados para CSV (útil para Power BI, Tableau, etc.)
output_csv = 'dados_marketing_consolidado.csv'
df_consolidado.to_csv(output_csv, index=False, encoding='utf-8-sig')
print(f"[OK] Dados consolidados exportados para: {output_csv}")

# Exportar resumo de conversão
if 'df_conversao' in locals():
    df_conversao.to_csv('resumo_conversao_leads.csv', index=False, encoding='utf-8-sig')
    print(f"[OK] Resumo de conversão exportado para: resumo_conversao_leads.csv")

---
## Fim da Análise

**Arquivo gerado**: KPI_Marketing_Preparado.xlsx  
**Script de processamento**: preparar_dados_marketing.py  
**Notebook**: analise_marketing.ipynb  

Para dúvidas ou customizações adicionais, modifique as células acima conforme necessário.